In [1]:
# Download data from yfinance

# required libraries
import yfinance as yf
import pandas as pd

# Tickers
stocks = ["MSFT", "NVDA", "GOOGL", "AMZN", "META"]
market = "^GSPC"   # S&P 500

# Get monthly Open, High, Low, Close, Adj Close, Volume for stocks
stocks_monthly = yf.download(
    tickers=stocks,
    start="2020-01-01",    # start Jan 2020
    end="2025-10-31",     # end October 2025
    interval="1mo",     # monthly intervals
    group_by="ticker",  # keeps each ticker separate
    auto_adjust=False   # keep raw OHLCV (not adjusted)
)

# Get monthly Open, High, Low, Close, Adj Close, Volume for market index
market_monthly = yf.download(
    tickers=market,
    start="2020-01-01",  # start Jan 2020
    end="2025-10-31",    # end October 2025
    interval="1mo",      # monthly intervals
    group_by="ticker",
    auto_adjust=False
)


[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  1 of 1 completed


In [2]:
# Company Data

# Flatten MultiIndex columns in stocks_monthly
stocks_monthly.columns = [
    f"{col[0]}_{col[1]}" for col in stocks_monthly.columns.to_flat_index()
]

stocks_monthly.head()

,GOOGL_Open,GOOGL_High,GOOGL_Low,GOOGL_Close,GOOGL_Adj Close,GOOGL_Volume,AMZN_Open,AMZN_High,AMZN_Low,AMZN_Close,...,META_Low,META_Close,META_Adj Close,META_Volume,NVDA_Open,NVDA_High,NVDA_Low,NVDA_Close,NVDA_Adj Close,NVDA_Volume
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,67.420502,75.028999,67.324501,71.639000,71.148285,673594000,93.750000,102.786003,90.766998,100.435997,...,201.059998,201.910004,200.672913,347314600,5.96875,6.48750,5.78175,5.91075,5.884791,6125412000
2020-02-01,73.082497,76.537003,63.410500,66.962502,66.503815,830656000,100.529999,109.297501,90.556503,94.187500,...,181.820007,192.470001,191.290756,317547300,5.89250,7.90800,5.88625,6.75175,6.722099,11848652000
2020-03-01,67.569504,70.409500,50.443501,58.097500,57.699539,1570716000,95.324501,99.816498,81.301498,97.486000,...,137.100006,166.800003,165.778046,649467700,6.92250,7.12225,4.51700,6.59000,6.564983,15773952000
2020-04-01,56.200001,68.007500,53.754002,67.334999,66.873764,1124224000,96.648499,123.750000,94.457497,123.699997,...,150.830002,204.710007,203.455780,541334600,6.39125,7.60500,5.95975,7.30700,7.279262,11278304000
2020-05-01,66.204498,72.255501,64.800499,71.676003,71.185028,725130000,116.839996,126.272499,112.819000,122.118500,...,198.759995,225.089996,223.710876,509476500,7.10875,9.18175,7.02100,8.87550,8.841806,12548876000


In [3]:
# S&P 500 data
# Flatten MultiIndex columns in stocks_monthly
market_monthly.columns = [
    f"{col[0]}_{col[1]}" for col in market_monthly.columns.to_flat_index() ]

market_monthly.head()

,^GSPC_Open,^GSPC_High,^GSPC_Low,^GSPC_Close,^GSPC_Adj Close,^GSPC_Volume
Date,,,,,,
2020-01-01,3244.669922,3337.770020,3214.639893,3225.520020,3225.520020,77287980000
2020-02-01,3235.659912,3393.520020,2855.840088,2954.219971,2954.219971,84436590000
2020-03-01,2974.280029,3136.719971,2191.860107,2584.590088,2584.590088,162185380000
2020-04-01,2498.080078,2954.860107,2447.489990,2912.429932,2912.429932,123608160000
2020-05-01,2869.090088,3068.669922,2766.639893,3044.310059,3044.310059,107135190000


In [4]:
# loadsaved google trends dataset
trends_monthly = pd.read_csv("ai_trends_monthly.csv")
trends_monthly = trends_monthly.set_index('date')
trends_monthly.index = pd.to_datetime(trends_monthly.index)

trends_monthly.head()

,AI,Gemini,ChatGPT,Machine Learning,OpenAI,Meta AI,Microsoft AI,NVIDIA AI,Google AI,Amazon AI
date,,,,,,,,,,
2020-01-01,5,2,0,1,0,0,0,0,1,0
2020-02-01,5,2,0,1,0,0,0,0,1,0
2020-03-01,5,1,0,0,0,0,0,0,1,0
2020-04-01,6,2,0,0,0,0,0,0,1,0
2020-05-01,5,2,0,1,0,0,0,0,1,0


In [9]:
# Compute Engineered features

# 1) Monthly returns for each stock
stocks_monthly_returns = stocks_monthly[['AMZN_Adj Close','META_Adj Close','GOOGL_Adj Close','NVDA_Adj Close','MSFT_Adj Close']].pct_change().dropna()
stocks_monthly_returns.columns = ['AMZN_ret','META_ret','GOOGL_ret','NVDA_ret','MSFT_ret']

# 2) Market index monthly returns
market_monthly_returns = market_monthly[['^GSPC_Adj Close']].pct_change().dropna()
market_monthly_returns.columns = ['Market_ret']

# 3) Monthly excess returns of each stock
excess_returns = stocks_monthly_returns.sub(market_monthly_returns['Market_ret'], axis=0)
excess_returns.columns = ['AMZN_excess','META_excess','GOOGL_excess','NVDA_excess','MSFT_excess']


# 4) Lagged Trends Features (t−1, t−2), for time-series analysis
trends_lagged = trends_monthly.copy()
lag_periods = [1, 2]

for col in trends_lagged.columns:
    for lag in lag_periods:
        trends_lagged[f'{col}_lag{lag}'] = trends_lagged[col].shift(lag)
        
# drop rows with NaN
trends_lagged = trends_lagged.dropna()


In [10]:
# Merge to final Data Frame
final_df = pd.concat([stocks_monthly_returns, excess_returns, trends_lagged], axis=1).dropna()
final_df

,AMZN_ret,META_ret,GOOGL_ret,NVDA_ret,MSFT_ret,AMZN_excess,META_excess,GOOGL_excess,NVDA_excess,MSFT_excess,...,Meta AI_lag1,Meta AI_lag2,Microsoft AI_lag1,Microsoft AI_lag2,NVIDIA AI_lag1,NVIDIA AI_lag2,Google AI_lag1,Google AI_lag2,Amazon AI_lag1,Amazon AI_lag2
2020-03-01,0.035021,-0.133371,-0.132388,-0.023373,-0.023883,0.160140,-0.008252,-0.007268,0.101746,0.101236,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2020-04-01,0.268900,0.227278,0.159000,0.108801,0.136327,0.142056,0.100434,0.032156,-0.018043,0.009483,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2020-05-01,-0.012785,0.099555,0.064469,0.214657,0.022543,-0.058067,0.054273,0.019187,0.169375,-0.022739,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2020-06-01,0.129567,0.008797,-0.010792,0.070109,0.113652,0.111178,-0.009592,-0.029180,0.051720,0.095264,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2020-07-01,0.147114,0.117144,0.049293,0.118117,0.007370,0.092012,0.062043,-0.005808,0.063016,-0.047731,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-01,0.070143,0.139925,0.026144,0.169170,0.082464,0.020536,0.090318,-0.023463,0.119564,0.032857,...,7.0,9.0,3.0,3.0,1.0,1.0,23.0,18.0,1.0,1.0
2025-07-01,0.067095,0.048700,0.090236,0.125909,0.072556,0.045429,0.027034,0.068570,0.104242,0.050889,...,6.0,7.0,3.0,3.0,1.0,1.0,31.0,23.0,2.0,1.0
2025-08-01,-0.021827,-0.044916,0.109484,-0.020746,-0.050253,-0.040894,-0.063983,0.090418,-0.039812,-0.069319,...,6.0,6.0,4.0,3.0,2.0,1.0,30.0,31.0,3.0,2.0
2025-09-01,-0.041179,-0.005848,0.141797,0.071191,0.023903,-0.076503,-0.041172,0.106473,0.035867,-0.011421,...,11.0,6.0,10.0,4.0,9.0,2.0,45.0,30.0,13.0,3.0


In [11]:
# Save data frames
stocks_monthly.to_csv('stocks_monthly.csv')
market_monthly.to_csv('market_monthly.csv')
trends_monthly.to_csv('trends_monthly.csv')

# computed dataframe 
final_df.to_csv('final_features.csv')